In [1]:
import numpy as np
np.seterr(divide='ignore') # happy with log(0) = -inf
from math import log, pi

class NBC:
    def __init__(self, feature_types, num_classes):
        self.feature_types = feature_types
        self.num_features = len(feature_types)
        self.num_classes = int(num_classes)        
    
    def fit(self, X, y):
        if hasattr(self, "logpi"):
            raise Exception("Already fitted")
        if X.shape[1] != self.num_features:
            raise Exception("Wrong number of features")
                    
        self.classes = np.unique(y)
        if len(self.classes) != self.num_classes:
            raise Execption("Wrong number of classes")
        self.class_map = {v: k[0] for k, v in np.ndenumerate(self.classes)}
                    
        self.logpi = np.log([np.count_nonzero(y == c) for c in self.classes]) - log(y.shape[0])
        
        self.means = []
        self.stddevs = []
        for i in range(0, self.num_classes):
            X_ = X[np.where(y == self.class_map[i])]
            self.means.append(np.mean(X_, axis = 0))
            self.stddevs.append(np.std(X_, axis = 0))
        self.means = np.array(self.means).T
        self.stddevs = np.array(self.stddevs).T
        
    def predict(self, X):
        if X.shape[1] != self.num_features:
            raise Exception("Wrong number of features")        
        
        X = np.repeat(np.expand_dims(X, axis=2), self.num_classes, axis=2)
        # X.shape == (data, features, classes)
        
        for class_ in range(self.num_classes):
            bernoulli_features = np.where(self.feature_types == 'b')[0]
            if bernoulli_features.shape[0] > 0:
                # Set X[data, feature, class] = log(P(X[data, feature, class) | mean[feature, class]))
                # for Bernoulli features using the pmf
                mean = self.means[bernoulli_features, class_]
                X[:, bernoulli_features, class_] = X[:, bernoulli_features, class_]*np.log(mean) + (1-X[:, bernoulli_features, class_]) * np.log(1-mean)
            gaussian_features = np.where(self.feature_types == 'r')[0]
            if gaussian_features.shape[0] > 0:
                mean = self.means[gaussian_features, class_]
                stddev = self.stddevs[gaussian_features, class_]
                # Set X[data, feature, class] = log(P(X[data, feature, class) | mean[feature, class], stddev[feature, class]))
                # for Normal features using the pdf
                X[:, gaussian_features, class_] = -1/2*np.log(2*pi*stddev) - (X[:, gaussian_features, class_]-mean)**2/2/stddev 
        
        X = np.sum(X, axis = 1) + self.logpi
        # X[data, class] = log(P(data | class))

        y = np.argmax(X, axis=1)
        # y[data] == argmax_c P(data | class)
                
        return self.classes[y]

In [2]:
def partition(X, y, factor=0.8):
    N, D = X.shape
    N_train = int(factor * N)
    return X[:N_train], y[:N_train], X[N_train:], y[N_train:]

report_string = "{}\nTrain: {:.5f}\nTest:  {:.5f}\n"

def compare(methods, X, y):
    X_train, y_train, X_test, y_test = partition(X, y)
    for (method, classifier) in methods:
        classifier.fit(X_train, y_train)
        train_accuracy = np.mean(classifier.predict(X_train) == y_train)
        test_accuracy = np.mean(classifier.predict(X_test) == y_test)
        print report_string.format(method, train_accuracy, test_accuracy)

In [3]:
from sklearn.naive_bayes import BernoulliNB 
import cPickle as cp

X, y = cp.load(open('data/voting.cPickle', 'rb'))
compare([("My NBC", NBC(np.repeat('b', 16), 2)), ("SKLearn",BernoulliNB())], X, y)

My NBC
Train: 0.96757
Test:  0.97872

SKLearn
Train: 0.90270
Test:  0.97872



/Users/rb/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:46: RuntimeWarning: invalid value encountered in multiply


In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris

iris = load_iris()
X, y = iris['data'], iris['target']

compare([(" My NBC", NBC(np.repeat('r', 4), 3)), ("SKLearn", LogisticRegression())], X, y)

 My NBC
Train: 0.95833
Test:  0.66667

SKLearn
Train: 0.92500
Test:  0.43333

